<a href="https://colab.research.google.com/github/mariobecerra/mda_project/blob/main/code/01_weekly_temperature_belgium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

URLs of data

In [2]:
arrondissements_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/arrondissements_coords.csv'
mortality_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/mortality_data_2000-2019.csv'
temperature_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/temp_2000_2019.csv'


Read data

In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
arrondissements_data = pd.read_csv(arrondissements_url)

arrondissements_data.head(5)

,NIS_Code,Nom_arrondissement,lat,lon
0,11000,Antwerpen,51.280681,4.505809
1,63000,Verviers,50.464213,6.022070
2,13000,Turnhout,51.250094,4.950422
3,62000,Liège,50.613433,5.607778
4,64000,Waremme,50.662789,5.206501


In [5]:
mortality_data = pd.read_csv(mortality_url)
mortality_data


,YEAR,YEAR_WEEK,COD,ARRON,N_MASK
0,2000,2000-001,external,11000,NaN
1,2000,2000-001,external,12000,NaN
2,2000,2000-001,external,13000,NaN
3,2000,2000-001,external,21000,NaN
4,2000,2000-001,external,23000,NaN
...,...,...,...,...,...
82579,2019,2019-053,natural,84000,6.0
82580,2019,2019-053,natural,85000,5.0
82581,2019,2019-053,natural,91000,10.0
82582,2019,2019-053,natural,92000,19.0


In [6]:
temperature_data = pd.read_csv(temperature_url, sep=";")
temperature_data

,GRID_NO,LATITUDE,LONGITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,96095,49.59159,5.03811,20000101,6.2,2.9,4.5
1,96095,49.59159,5.03811,20000102,6.0,4.2,5.1
2,96095,49.59159,5.03811,20000103,5.4,3.1,4.3
3,96095,49.59159,5.03811,20000104,8.1,4.4,6.3
4,96095,49.59159,5.03811,20000105,8.1,1.8,4.9
...,...,...,...,...,...,...,...
540565,105095,51.60877,4.81995,20191227,6.6,5.5,6.0
540566,105095,51.60877,4.81995,20191228,5.3,-1.1,2.1
540567,105095,51.60877,4.81995,20191229,5.0,-2.7,1.1
540568,105095,51.60877,4.81995,20191230,9.5,0.9,5.2


Find the closest temperature grid to each arrondissement

In [7]:
grids_temp = temperature_data.drop_duplicates(subset = ['GRID_NO', 'LATITUDE', 'LONGITUDE'])
grids_temp

,GRID_NO,LATITUDE,LONGITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,96095,49.59159,5.03811,20000101,6.2,2.9,4.5
7305,96096,49.60623,5.38328,20000101,5.8,2.4,4.1
14610,96097,49.61982,5.72863,20000101,4.9,1.1,3.0
21915,96098,49.63236,6.07415,20000101,5.3,1.5,3.4
29220,97095,49.81581,5.01500,20000101,5.9,2.5,4.2
...,...,...,...,...,...,...,...
504045,104094,51.36832,4.48712,20000101,8.5,4.9,6.7
511350,104095,51.38472,4.84537,20000101,8.1,4.5,6.3
518655,104096,51.40002,5.20384,20000101,7.9,4.0,6.0
525960,104097,51.41421,5.56253,20000101,7.9,4.0,6.0


In [8]:
# Not the most efficient, but it's a small dataset so it's okay
distances = pd.DataFrame()

for i in range(arrondissements_data.shape[0]):
  diff_lat = grids_temp['LATITUDE'] - arrondissements_data.loc[i].at['lat']
  diff_lon = grids_temp['LONGITUDE'] - arrondissements_data.loc[i].at['lon']
  distances_i = np.power((diff_lat), 2) + np.power((diff_lon), 2)

  temp = pd.DataFrame(
        {
            'NIS_Code': arrondissements_data.loc[i].at['NIS_Code'],
            'GRID_NO': grids_temp.GRID_NO,
            'Dist': distances_i
        }
    )
  distances = pd.concat([distances, temp])



In [9]:
distances

,NIS_Code,GRID_NO,Dist
0,11000,96095,3.136373
7305,11000,96096,3.573742
14610,11000,96097,4.253751
21915,11000,96098,5.176657
29220,11000,97095,2.405123
...,...,...,...
504045,54000,104094,2.017259
511350,54000,104095,3.076978
518655,54000,104096,4.393328
525960,54000,104097,5.966667


Find the temperature grid closest to each arrondissement

In [10]:
arron_grid_mapping = distances.groupby('NIS_Code').apply(lambda x: x[x['Dist'] == x['Dist'].min()])[['NIS_Code', 'GRID_NO']].reset_index(drop = True)

arron_grid_mapping.head(8)

,NIS_Code,GRID_NO
0,11000,104094
1,12000,103094
2,13000,103095
3,21000,102094
4,23000,102093
5,24000,102095
6,25000,101094
7,31000,104090


Join arrondissement and temperature datasets to get temperature per day in each arrondissement.

In [11]:
join_1 = pd.merge(arron_grid_mapping, temperature_data, how = "inner")
join_1


,NIS_Code,GRID_NO,LATITUDE,LONGITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,11000,104094,51.36832,4.48712,20000101,8.5,4.9,6.7
1,11000,104094,51.36832,4.48712,20000102,9.5,5.4,7.4
2,11000,104094,51.36832,4.48712,20000103,9.8,7.6,8.7
3,11000,104094,51.36832,4.48712,20000104,9.8,7.2,8.5
4,11000,104094,51.36832,4.48712,20000105,8.1,3.2,5.7
...,...,...,...,...,...,...,...,...
321415,93000,99094,50.24822,4.61816,20191227,6.7,4.2,5.4
321416,93000,99094,50.24822,4.61816,20191228,4.6,3.8,4.2
321417,93000,99094,50.24822,4.61816,20191229,7.2,-1.8,2.7
321418,93000,99094,50.24822,4.61816,20191230,9.2,1.1,5.2


In [12]:
arron_temp_data = pd.merge(join_1, arrondissements_data, how = 'left')[['DAY', 'Nom_arrondissement', 'NIS_Code', 'GRID_NO', 'TEMPERATURE_MAX', 'TEMPERATURE_MIN', 'TEMPERATURE_AVG']].sort_values(by=['Nom_arrondissement', 'NIS_Code', 'DAY']).assign(YEAR = lambda x: np.floor(x.DAY/10000))
arron_temp_data

# Saving arron_emp_data because it will be used for meta-analysis dataset construction
arron_temp_data.to_csv('out/arron_temp_data.csv', encoding = 'utf-8-sig') 


Get percentiles to define heatwave

In [14]:
def q95(x):
    return x.quantile(0.95)

temp_percetiles_year_arron = arron_temp_data[['YEAR', 'NIS_Code', 'Nom_arrondissement', 'TEMPERATURE_MAX']].groupby(['YEAR', 'NIS_Code', 'Nom_arrondissement']).agg(q95).stack(level=0).reset_index().rename(columns={0:"p95"})
temp_percetiles_year_arron




,YEAR,NIS_Code,Nom_arrondissement,level_3,p95
0,2000.0,11000,Antwerpen,TEMPERATURE_MAX,22.25
1,2000.0,12000,Mechelen,TEMPERATURE_MAX,22.45
2,2000.0,13000,Turnhout,TEMPERATURE_MAX,22.40
3,2000.0,21000,Bruxelles-Capitale,TEMPERATURE_MAX,22.10
4,2000.0,23000,Hal-Vilvorde,TEMPERATURE_MAX,22.15
...,...,...,...,...,...
875,2019.0,84000,Neufchâteau,TEMPERATURE_MAX,22.80
876,2019.0,85000,Virton,TEMPERATURE_MAX,23.92
877,2019.0,91000,Dinant,TEMPERATURE_MAX,22.80
878,2019.0,92000,Namur,TEMPERATURE_MAX,23.30


Get temperatures of five consecutive days

In [15]:

temp_lead_1 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-1)

temp_lead_2 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-2)

temp_lead_3 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-3)

temp_lead_4 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-4)

    

In [16]:
arron_temp_data['temp_lead_1'] = temp_lead_1
arron_temp_data['temp_lead_2'] = temp_lead_2
arron_temp_data['temp_lead_3'] = temp_lead_3
arron_temp_data['temp_lead_4'] = temp_lead_4
arron_temp_data

,DAY,Nom_arrondissement,NIS_Code,GRID_NO,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG,YEAR,temp_lead_1,temp_lead_2,temp_lead_3,temp_lead_4
116880,20000101,Aalst,41000,102092,8.4,4.7,6.6,2000.0,8.8,9.5,10.2,7.2
116881,20000102,Aalst,41000,102092,8.8,5.7,7.3,2000.0,9.5,10.2,7.2,9.0
116882,20000103,Aalst,41000,102092,9.5,7.1,8.3,2000.0,10.2,7.2,9.0,6.6
116883,20000104,Aalst,41000,102092,10.2,7.1,8.7,2000.0,7.2,9.0,6.6,8.7
116884,20000105,Aalst,41000,102092,7.2,2.1,4.7,2000.0,9.0,6.6,8.7,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...
80350,20191227,Ypres,33000,102089,8.0,6.9,7.4,2019.0,7.5,4.8,7.7,5.8
80351,20191228,Ypres,33000,102089,7.5,5.7,6.6,2019.0,4.8,7.7,5.8,NaN
80352,20191229,Ypres,33000,102089,4.8,0.5,2.7,2019.0,7.7,5.8,NaN,NaN
80353,20191230,Ypres,33000,102089,7.7,1.2,4.5,2019.0,5.8,NaN,NaN,NaN


Create boolean variable if there are 5 consecutive days in which temperature was higher than the 95-th percentile of temperature each year and each arrondissement

In [17]:
join_percentiles = pd.merge(arron_temp_data, temp_percetiles_year_arron[['YEAR', 'NIS_Code', 'Nom_arrondissement', 'p95']], how = "left")
join_percentiles

,DAY,Nom_arrondissement,NIS_Code,GRID_NO,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG,YEAR,temp_lead_1,temp_lead_2,temp_lead_3,temp_lead_4,p95
0,20000101,Aalst,41000,102092,8.4,4.7,6.6,2000.0,8.8,9.5,10.2,7.2,22.20
1,20000102,Aalst,41000,102092,8.8,5.7,7.3,2000.0,9.5,10.2,7.2,9.0,22.20
2,20000103,Aalst,41000,102092,9.5,7.1,8.3,2000.0,10.2,7.2,9.0,6.6,22.20
3,20000104,Aalst,41000,102092,10.2,7.1,8.7,2000.0,7.2,9.0,6.6,8.7,22.20
4,20000105,Aalst,41000,102092,7.2,2.1,4.7,2000.0,9.0,6.6,8.7,6.7,22.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321415,20191227,Ypres,33000,102089,8.0,6.9,7.4,2019.0,7.5,4.8,7.7,5.8,23.14
321416,20191228,Ypres,33000,102089,7.5,5.7,6.6,2019.0,4.8,7.7,5.8,NaN,23.14
321417,20191229,Ypres,33000,102089,4.8,0.5,2.7,2019.0,7.7,5.8,NaN,NaN,23.14
321418,20191230,Ypres,33000,102089,7.7,1.2,4.5,2019.0,5.8,NaN,NaN,NaN,23.14


In [19]:
bool0 = (join_percentiles['TEMPERATURE_MAX'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool1 = (join_percentiles['temp_lead_1'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool2 = (join_percentiles['temp_lead_2'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool3 = (join_percentiles['temp_lead_3'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool4 = (join_percentiles['temp_lead_4'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
heatwave_boolean = bool0 * bool1 * bool2 * bool3 * bool4
heatwave_boolean

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
# Not the most efficient, but it works
iso_weeks = ['1990-001']*join_percentiles.shape[0] # Prefill list with correct size
for i in range(len(iso_weeks)):
  day_i = join_percentiles.loc[i].at['DAY']
  iso_week_i = datetime.datetime.strptime(str(day_i), '%Y%m%d').isocalendar()
  iso_weeks[i] = str(iso_week_i[0]) + '-' + str(iso_week_i[1]).zfill(3)



In [21]:
heat_wave_day_def = join_percentiles[['NIS_Code', 'Nom_arrondissement']]
heat_wave_day_def['heatwave_boolean'] = heatwave_boolean
heat_wave_day_def['YEAR_WEEK'] = iso_weeks
heat_wave_day_def

<ipython-input-21-87b199a954f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_wave_day_def['heatwave_boolean'] = heatwave_boolean
<ipython-input-21-87b199a954f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_wave_day_def['YEAR_WEEK'] = iso_weeks


,NIS_Code,Nom_arrondissement,heatwave_boolean,YEAR_WEEK
0,41000,Aalst,0,1999-052
1,41000,Aalst,0,1999-052
2,41000,Aalst,0,2000-001
3,41000,Aalst,0,2000-001
4,41000,Aalst,0,2000-001
...,...,...,...,...
321415,33000,Ypres,0,2019-052
321416,33000,Ypres,0,2019-052
321417,33000,Ypres,0,2019-052
321418,33000,Ypres,0,2020-001


In [22]:
heat_wave_week = heat_wave_day_def.groupby(['NIS_Code', 'Nom_arrondissement', 'YEAR_WEEK'])['heatwave_boolean'].sum().reset_index()
heat_wave_week['heatwave_week_boolean'] = (heat_wave_week['heatwave_boolean'] > 0).astype(int)
heat_wave_week

,NIS_Code,Nom_arrondissement,YEAR_WEEK,heatwave_boolean,heatwave_week_boolean
0,11000,Antwerpen,1999-052,0,0
1,11000,Antwerpen,2000-001,0,0
2,11000,Antwerpen,2000-002,0,0
3,11000,Antwerpen,2000-003,0,0
4,11000,Antwerpen,2000-004,0,0
...,...,...,...,...,...
45975,93000,Philippeville,2019-049,0,0
45976,93000,Philippeville,2019-050,0,0
45977,93000,Philippeville,2019-051,0,0
45978,93000,Philippeville,2019-052,0,0


In [23]:
heat_wave_week['heatwave_week_boolean'].sum()

854

In [24]:
mortality_heat_wave = pd.merge(heat_wave_week[['YEAR_WEEK', 'NIS_Code', 'Nom_arrondissement', 'heatwave_week_boolean']], mortality_data[['ARRON', 'YEAR_WEEK', 'COD', 'N_MASK']], left_on = ['NIS_Code', 'YEAR_WEEK'], right_on = ['ARRON', 'YEAR_WEEK'], how = 'inner')
mortality_heat_wave

,YEAR_WEEK,NIS_Code,Nom_arrondissement,heatwave_week_boolean,ARRON,COD,N_MASK
0,2000-001,11000,Antwerpen,0,11000,external,NaN
1,2000-001,11000,Antwerpen,0,11000,natural,31.0
2,2000-002,11000,Antwerpen,0,11000,external,13.0
3,2000-002,11000,Antwerpen,0,11000,natural,263.0
4,2000-003,11000,Antwerpen,0,11000,external,11.0
...,...,...,...,...,...,...,...
81137,2019-050,93000,Philippeville,0,93000,external,NaN
81138,2019-050,93000,Philippeville,0,93000,natural,13.0
81139,2019-051,93000,Philippeville,0,93000,natural,15.0
81140,2019-052,93000,Philippeville,0,93000,external,NaN


Save dataset

In [25]:
import os
directory = "out/"
if not os.path.exists(directory):
    os.makedirs(directory)

In [26]:
mortality_heat_wave.to_csv('out/mortality_heat_wave.csv', encoding = 'utf-8-sig') 

In [27]:
## Only run if running on Google Colab
# from google.colab import files
# files.download('out/mortality_heat_wave.csv')